<a href="https://colab.research.google.com/github/m25c1049/generative_AI/blob/main/Toyama_Uni_LangChain_Llama2_7b_Q_RAG_Renovated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Application Sample Code :
##Structure
- LLM: mmnga/ELYZA-japanese-Llama-2-7b-instruct-gguf on Hugging-Face
- Embedding: sentence-transformers/distiluse-base-multilingual-cased-v2
- RAG(VectorDB): Chroma
- RAG Data Source: https://www.aozora.gr.jp/cards/000081/files/43754_17659.html

##Environment Resource Requirement
- RAM: 13GB over
- Disc: 50GB Free

##Detail
- Install libraris by pip command.
- Embedding Model Configuration.
- Scraping data from Web Site.
- Splited loaded text and create chunk data.
- Insert chunk data into Vector DB.
- Configure Vector DB as Retriever(RAG).
- Create Prompt as ChatPromptTemplate instance.
- Define Lang Chain Expression Language (LCEL).
- Execute RAG retrieval, context injection to prompt, question injection to prompt, LLM execution, and get response as stream data.

In [1]:
!pip install langchain
!pip install langchain-core
!pip install langchain-community
!pip install langchain-huggingface
!pip install llama-cpp-python
!pip install huggingface-hub
!pip install sentence-transformers
!pip install chromadb
!pip install beautifulsoup4
!pip install lxml
!pip install requests
!pip install numpy
!pip install transformers
!pip install torch
!pip install tokenizers
# google-colab の依存関係を解決するため、requests==2.32.4 を明示的にインストールします。
# これにより、langchain-community に関連する警告が表示される可能性があります。
!pip install requests==2.32.4 --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.8 MB/s eta 0:

In [2]:
from huggingface_hub import hf_hub_download
from langchain_community.llms import LlamaCpp
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_huggingface import HuggingFaceEmbeddings
import requests
from bs4 import BeautifulSoup

CONTEXT_SIZE = 2048
LLM_REPO_ID = "mmnga/ELYZA-japanese-Llama-2-7b-instruct-gguf"
LLM_FILE = "ELYZA-japanese-Llama-2-7b-instruct-q4_K_S.gguf"
CHUNK_SIZE = 256
CHUNK_OVERLAP = 64
EMB_MODEL = "sentence-transformers/distiluse-base-multilingual-cased-v2"
COLLECTION_NAME = "langchain"
SRC_INFO_URL = "https://www3.nhk.or.jp/news/word/0001813.html"

# LLMを生成
model_path = hf_hub_download(repo_id=LLM_REPO_ID, filename=LLM_FILE)
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=128,
    n_ctx=CONTEXT_SIZE,
    f16_kv=True,
    verbose=True,
    seed=0
)

# 埋め込み表現生成用モデルを準備
embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL)

# 指定したURLから情報ソースをロードし、目的のリンクを抽出
response = requests.get(SRC_INFO_URL)
response.encoding = response.apparent_encoding
soup = BeautifulSoup(response.text, 'lxml')

# "【演説全文】****氏 自民党総裁選挙" を含むリンクを検索
links = []
for a_tag in soup.find_all('a', href=True):
    if "【演説全文】" in a_tag.get_text() and "氏 自民党総裁選挙" in a_tag.get_text():
        # 相対URLを絶対URLに変換
        link = requests.compat.urljoin(SRC_INFO_URL, a_tag['href'])
        links.append(link)

# 抽出したリンク先のコンテンツをロード
all_data = []
for link in links:
    try:
        loader = WebBaseLoader(link)
        all_data.extend(loader.load())
    except Exception as e:
        print(f"Error loading {link}: {e}")


# ロードしたテキストをチャンクに分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP
)
all_splits = text_splitter.split_documents(all_data)

# ベクトル化してベクトルDBへ格納
vector_store = Chroma.from_documents(
    documents=all_splits, embedding=embeddings
)

# ベクトルDBをLangChainのRetrieverに設定、抽出するチャンク数はkで設定
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

# Llama2プロンプトテンプレート
template = """<s>[INST] <<SYS>>
あなたは優秀で中立的な日本人の政治学者です。前提条件の情報だけで回答してください。
<</SYS>>

前提条件：{context}

質問：{question} [/INST]"""

# LangChain LCELでチェインを構築
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | llm | output_parser

# ユーザーが質問を入力
question = input("質問を入力してください: ")

# チェインを起動して、回答をストリーミング出力
for s in chain.stream(question):
    print(s, end="", flush=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


(…)japanese-Llama-2-7b-instruct-q4_K_S.gguf:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--mmnga--ELYZA-japanese-Llama-2-7b-instruct-gguf/snapshots/2d708f9c52bde588049a494e95b986f5bedba76f/ELYZA-japanese-Llama-2-7b-instruct-q4_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = ELYZA-japanese-Llama-2-7b-instruct
llama_model_loader: - kv   2:       general.source.hugginface.repository str              = elyza/ELYZA-japanese-Llama-2-7b-instruct
llama_model_loader: - kv   3:                   llama.tensor_data_layout str              = Meta AI original pth
llama_model_loader: - kv   4:                       llama.context_length u32              = 4096
llama_model_loader: - kv   5:                     ll

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

質問を入力してください: 外国人政策の面で5人を比較してください。
  承知しました。外国人政策の面で、林芳正氏と小林鷹之氏を比較いたします。

- 林芳正氏: 強化
- 小林鷹之氏: 厳格化

llama_perf_context_print:        load time =  431064.11 ms
llama_perf_context_print: prompt eval time =  431061.80 ms /  1144 tokens (  376.80 ms per token,     2.65 tokens per second)
llama_perf_context_print:        eval time =   59399.06 ms /    80 runs   (  742.49 ms per token,     1.35 tokens per second)
llama_perf_context_print:       total time =  490648.92 ms /  1224 tokens
llama_perf_context_print:    graphs reused =        183
